# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
# Import pustaka yang dibutuhkan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Pustaka untuk klasifikasi
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Pustaka untuk validasi model
from sklearn.model_selection import cross_val_score


# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [5]:
import pandas as pd

# Memuat dataset
df = pd.read_csv('bank.csv')

# Pastikan bahwa dataset telah dimuat dengan benar
print(df.head())


  TransactionID AccountID  TransactionAmount      TransactionDate  \
0      TX000001   AC00128              14.09  2023-04-11 16:29:14   
1      TX000002   AC00455             376.24  2023-06-27 16:44:19   
2      TX000003   AC00019             126.29  2023-07-10 18:16:08   
3      TX000004   AC00070             184.50  2023-05-05 16:32:11   
4      TX000005   AC00411              13.45  2023-10-16 17:51:24   

  TransactionType   Location DeviceID      IP Address MerchantID Channel  \
0           Debit  San Diego  D000380  162.198.218.92       M015     ATM   
1           Debit    Houston  D000051     13.149.61.4       M052     ATM   
2           Debit       Mesa  D000235  215.97.143.157       M009  Online   
3           Debit    Raleigh  D000187  200.13.225.150       M002  Online   
4          Credit    Atlanta  D000308    65.164.3.100       M091  Online   

   CustomerAge CustomerOccupation  TransactionDuration  LoginAttempts  \
0           70             Doctor                   81 

# **3. Data Splitting**

In [10]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Menyiapkan fitur numerik
numerical_cols = ['TransactionAmount', 'TransactionDuration', 'CustomerAge', 'LoginAttempts', 'AccountBalance']

# Menstandarisasi data numerik
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[numerical_cols])

# Melatih model KMeans dengan jumlah cluster yang optimal
kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10, random_state=42)
df['Cluster'] = kmeans.fit_predict(df_scaled)

# Menampilkan dataset setelah penambahan cluster
print(df.head())


  TransactionID AccountID  TransactionAmount      TransactionDate  \
0      TX000001   AC00128              14.09  2023-04-11 16:29:14   
1      TX000002   AC00455             376.24  2023-06-27 16:44:19   
2      TX000003   AC00019             126.29  2023-07-10 18:16:08   
3      TX000004   AC00070             184.50  2023-05-05 16:32:11   
4      TX000005   AC00411              13.45  2023-10-16 17:51:24   

  TransactionType   Location DeviceID      IP Address MerchantID Channel  \
0           Debit  San Diego  D000380  162.198.218.92       M015     ATM   
1           Debit    Houston  D000051     13.149.61.4       M052     ATM   
2           Debit       Mesa  D000235  215.97.143.157       M009  Online   
3           Debit    Raleigh  D000187  200.13.225.150       M002  Online   
4          Credit    Atlanta  D000308    65.164.3.100       M091  Online   

   CustomerAge CustomerOccupation  TransactionDuration  LoginAttempts  \
0           70             Doctor                   81 

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Memisahkan fitur dan target
X = df.drop(columns=['TransactionID', 'AccountID', 'TransactionDate', 'PreviousTransactionDate', 'Cluster'])  # Fitur
y = df['Cluster']  # Target yang ingin diprediksi, menggunakan label cluster dari clustering

# Menangani fitur kategorikal dengan LabelEncoder untuk kolom yang tidak numerik
categorical_cols = X.select_dtypes(include=['object']).columns

label_encoder = LabelEncoder()
for col in categorical_cols:
    X[col] = label_encoder.fit_transform(X[col])

# Pastikan tidak ada nilai kosong (NaN) dalam data
X = X.fillna(0)
y = y.fillna(0)

# Membagi data menjadi data latih (training set) dan data uji (test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Menampilkan ukuran dataset latih dan uji
print(f"Jumlah data latih: {X_train.shape[0]}")
print(f"Jumlah data uji: {X_test.shape[0]}")


Jumlah data latih: 2009
Jumlah data uji: 503


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Membuat model klasifikasi Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Melatih model dengan data latih
rf_classifier.fit(X_train, y_train)

# Prediksi pada data uji
y_pred = rf_classifier.predict(X_test)

# Evaluasi model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.9840954274353877

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       386
           1       0.94      0.98      0.96       101
           2       1.00      1.00      1.00        16

    accuracy                           0.98       503
   macro avg       0.98      0.99      0.98       503
weighted avg       0.98      0.98      0.98       503


Confusion Matrix:
[[380   6   0]
 [  2  99   0]
 [  0   0  16]]


Jika Anda ingin menggunakan algoritma lain (seperti Logistic Regression atau K-Nearest Neighbors), Anda bisa mengganti RandomForestClassifier dengan algoritma yang Anda pilih. Misalnya, untuk Logistic Regression:

In [13]:
from sklearn.linear_model import LogisticRegression

# Membuat model klasifikasi Logistic Regression
log_reg = LogisticRegression(random_state=42)

# Melatih model dengan data latih
log_reg.fit(X_train, y_train)

# Prediksi pada data uji
y_pred_log_reg = log_reg.predict(X_test)

# Evaluasi model
print("Accuracy:", accuracy_score(y_test, y_pred_log_reg))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_log_reg))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_log_reg))


Accuracy: 0.9224652087475149

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96       386
           1       0.83      0.90      0.87       101
           2       0.00      0.00      0.00        16

    accuracy                           0.92       503
   macro avg       0.59      0.62      0.61       503
weighted avg       0.89      0.92      0.91       503


Confusion Matrix:
[[373  13   0]
 [ 10  91   0]
 [ 11   5   0]]


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no pre

Random Forest Classifier:Hasil Evaluasi Model:

Accuracy: 98.41%
Classification Report:

Precision untuk Cluster 0: 0.99
Precision untuk Cluster 1: 0.94
Precision untuk Cluster 2: 1.00
Confusion Matrix:
Menunjukkan bahwa model menghasilkan sebagian besar prediksi yang benar untuk semua kelas, dengan sebagian besar kesalahan terjadi antara Cluster 0 dan Cluster 1.

Menggunakan Algoritma Lain: Logistic Regression, Hasil Evaluasi Model:

Accuracy: 92.25%
Classification Report:

Precision untuk Cluster 0: 0.95
Precision untuk Cluster 1: 0.83
Precision untuk Cluster 2: 0.00 (karena tidak ada prediksi untuk kelas ini)
Confusion Matrix:

Cluster 0 cenderung diprediksi dengan sangat baik, namun ada sejumlah kesalahan dalam memprediksi Cluster 1 dan Cluster 2.
Penjelasan Mengenai Hasil:
Accuracy: Mengukur seberapa sering model menghasilkan prediksi yang benar, dengan Random Forest memberikan hasil yang lebih baik (98.41%) dibandingkan Logistic Regression (92.25%).
Classification Report: Memperlihatkan precision, recall, dan F1-score untuk setiap kelas. Random Forest lebih unggul dalam mengklasifikasikan semua kelas, sementara Logistic Regression mengalami kesulitan dalam mengklasifikasikan Cluster 2 dengan benar.
Confusion Matrix: Memberikan gambaran visual dari kesalahan prediksi. Meskipun Random Forest menghasilkan sedikit kesalahan, Logistic Regression membuat lebih banyak kesalahan, terutama dalam mengklasifikasikan Cluster 2.


Kesimpulan:
Random Forest Classifier memberikan performa yang lebih baik dibandingkan dengan Logistic Regression dalam hal klasifikasi data ini. Model Random Forest menunjukkan hasil yang lebih baik dalam mengklasifikasikan semua kelas, dengan tingkat akurasi yang sangat tinggi dan F1-score yang lebih seimbang.
Logistic Regression menghasilkan akurasi yang lebih rendah dan mengalami kesulitan dalam mengklasifikasikan kelas minoritas (Cluster 2), yang menunjukkan bahwa model tersebut mungkin tidak cukup kompleks untuk menangani data ini secara optimal.
Dari hasil tersebut, Random Forest adalah pilihan yang lebih baik untuk klasifikasi data ini.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

1. Prediksi Menggunakan Data Uji
Setelah model dilatih dengan data latih, langkah berikutnya adalah melakukan prediksi terhadap data uji. Dengan data uji, kita dapat mengukur seberapa baik model tersebut dalam membuat prediksi.

2. Metrik Evaluasi: Accuracy, F1-Score, Precision, dan Recall
Evaluasi dilakukan menggunakan beberapa metrik utama yang dapat membantu menilai kualitas prediksi:

Accuracy: Proporsi prediksi yang benar dari total data.
Precision: Mengukur ketepatan prediksi model untuk kelas tertentu, yaitu seberapa banyak prediksi benar dibandingkan dengan jumlah semua prediksi kelas tersebut.
Recall: Mengukur kemampuan model untuk menangkap semua data yang benar untuk suatu kelas.
F1-Score: Rata-rata harmonik antara precision dan recall, memberikan gambaran tentang keseimbangan antara keduanya.
3. Confusion Matrix
Confusion Matrix memberikan informasi mendalam tentang prediksi benar dan salah untuk setiap kelas. Ini memperlihatkan bagaimana model membedakan antara kelas-kelas yang ada.

In [14]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Prediksi menggunakan data uji
y_pred = rf_classifier.predict(X_test)  # Untuk Random Forest, atau gunakan log_reg.predict(X_test) untuk Logistic Regression

# Evaluasi model: Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Evaluasi model: Classification Report (Precision, Recall, F1-Score)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Evaluasi model: Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.9840954274353877

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       386
           1       0.94      0.98      0.96       101
           2       1.00      1.00      1.00        16

    accuracy                           0.98       503
   macro avg       0.98      0.99      0.98       503
weighted avg       0.98      0.98      0.98       503


Confusion Matrix:
[[380   6   0]
 [  2  99   0]
 [  0   0  16]]


1. Hasil Evaluasi dengan Random Forest
Accuracy:

Accuracy: 0.98 (atau 98%)
Model Random Forest berhasil mengklasifikasikan data dengan tingkat keberhasilan yang sangat tinggi, yaitu sekitar 98% dari total prediksi.
Classification Report:

Precision, Recall, dan F1-Score untuk setiap kelas:
Cluster 0:
Precision: 0.99, Recall: 0.98, F1-Score: 0.99
Model sangat baik dalam mengenali kelas 0, dengan hanya sedikit kesalahan.
Cluster 1:
Precision: 0.94, Recall: 0.98, F1-Score: 0.96
Model berhasil mengenali kelas 1 dengan cukup baik, meskipun ada sedikit kesalahan dalam prediksi kelas ini.
Cluster 2:
Precision: 1.00, Recall: 1.00, F1-Score: 1.00
Model sempurna dalam mengenali kelas 2 dengan semua prediksi benar.
Average:
Macro avg: Precision 0.98, Recall 0.99, F1-Score 0.98
Weighted avg: Precision 0.98, Recall 0.98, F1-Score 0.98
Model ini menunjukkan performa yang sangat baik pada semua kelas, dengan nilai rata-rata yang sangat tinggi.


Cluster 0: 380 benar dan 6 salah (prediksi salah untuk kelas 0).
Cluster 1: 99 benar dan 2 salah (prediksi salah untuk kelas 1).
Cluster 2: 16 benar dan tidak ada kesalahan (100% prediksi benar untuk kelas 2).

2. Hasil Evaluasi dengan Logistic Regression
Accuracy:

Accuracy: 0.92 (atau 92%)
Model Logistic Regression menghasilkan akurasi 92%, yang lebih rendah dibandingkan dengan Random Forest.
Classification Report:

Precision, Recall, dan F1-Score untuk setiap kelas:
Cluster 0:
Precision: 0.95, Recall: 0.97, F1-Score: 0.96
Model cukup baik dalam mengenali kelas 0, meskipun ada beberapa kesalahan.
Cluster 1:
Precision: 0.83, Recall: 0.90, F1-Score: 0.87
Model cukup baik mengenali kelas 1, tetapi ada beberapa kesalahan yang lebih besar dibandingkan dengan Random Forest.
Cluster 2:
Precision: 0.00, Recall: 0.00, F1-Score: 0.00
Model gagal mengenali kelas 2, dengan semua prediksi salah.
Average:
Macro avg: Precision 0.59, Recall 0.62, F1-Score 0.61
Weighted avg: Precision 0.89, Recall 0.92, F1-Score 0.91
Meskipun rata-rata diukur secara keseluruhan cukup tinggi, nilai untuk Cluster 2 sangat rendah, yang mengindikasikan bahwa model kesulitan mengenali kelas 2.


Cluster 0: 373 benar dan 13 salah (prediksi salah untuk kelas 0).
Cluster 1: 91 benar dan 10 salah (prediksi salah untuk kelas 1).
Cluster 2: 0 benar dan 16 salah (model tidak dapat mengenali kelas 2 dengan baik).

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

1. Melakukan Tuning Model dengan GridSearchCV
a. Menyiapkan Parameter Grid untuk Random Forest

Kita akan mencoba beberapa kombinasi untuk hyperparameter berikut:

n_estimators (Jumlah pohon yang digunakan)
max_depth (Maksimal kedalaman pohon)
min_samples_split (Jumlah minimum sampel yang diperlukan untuk membagi node)
min_samples_leaf (Jumlah minimum sampel di daun)
b. Implementasi GridSearchCV

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Menyiapkan parameter grid untuk GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Membuat model RandomForest
rf_classifier = RandomForestClassifier(random_state=42)

# Menyusun GridSearchCV dengan 5-fold cross-validation
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Melatih model dengan GridSearchCV
grid_search.fit(X_train, y_train)

# Menampilkan hasil terbaik
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_}")

# Mendapatkan model terbaik dari grid search
best_rf_classifier = grid_search.best_estimator_

# Prediksi menggunakan model terbaik
y_pred_best = best_rf_classifier.predict(X_test)

# Evaluasi model terbaik
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_test, y_pred_best))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_best))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_best))


Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best cross-validation score: 0.9890485229711791
Accuracy: 0.9840954274353877

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       386
           1       0.94      0.98      0.96       101
           2       1.00      1.00      1.00        16

    accuracy                           0.98       503
   macro avg       0.98      0.99      0.98       503
weighted avg       0.98      0.98      0.98       503


Confusion Matrix:
[[380   6   0]
 [  2  99   0]
 [  0   0  16]]


Tuning dengan RandomizedSearchCV (Alternatif

In [16]:
from sklearn.model_selection import RandomizedSearchCV

# Menyusun parameter distribusi untuk RandomizedSearchCV
param_dist = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Membuat model RandomForest
rf_classifier = RandomForestClassifier(random_state=42)

# Menyusun RandomizedSearchCV dengan 5-fold cross-validation
random_search = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_dist,
                                   n_iter=10, cv=5, n_jobs=-1, verbose=2, random_state=42)

# Melatih model dengan RandomizedSearchCV
random_search.fit(X_train, y_train)

# Menampilkan hasil terbaik
print(f"Best parameters: {random_search.best_params_}")
print(f"Best cross-validation score: {random_search.best_score_}")

# Mendapatkan model terbaik dari random search
best_rf_classifier_random = random_search.best_estimator_

# Prediksi menggunakan model terbaik
y_pred_best_random = best_rf_classifier_random.predict(X_test)

# Evaluasi model terbaik
print("Accuracy:", accuracy_score(y_test, y_pred_best_random))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_best_random))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_best_random))


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': None}
Best cross-validation score: 0.9890485229711791
Accuracy: 0.9821073558648111

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       386
           1       0.94      0.97      0.96       101
           2       1.00      1.00      1.00        16

    accuracy                           0.98       503
   macro avg       0.98      0.98      0.98       503
weighted avg       0.98      0.98      0.98       503


Confusion Matrix:
[[380   6   0]
 [  3  98   0]
 [  0   0  16]]


## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [17]:
# Prediksi menggunakan model terbaik yang telah dituning
y_pred_best = best_rf_classifier.predict(X_test)

# Evaluasi model terbaik
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Hitung Accuracy
accuracy = accuracy_score(y_test, y_pred_best)

# Tampilkan Classification Report
class_report = classification_report(y_test, y_pred_best)

# Tampilkan Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_best)

# Menampilkan hasil evaluasi
print("Accuracy:", accuracy)
print("\nClassification Report:")
print(class_report)
print("\nConfusion Matrix:")
print(conf_matrix)


Accuracy: 0.9840954274353877

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       386
           1       0.94      0.98      0.96       101
           2       1.00      1.00      1.00        16

    accuracy                           0.98       503
   macro avg       0.98      0.99      0.98       503
weighted avg       0.98      0.98      0.98       503


Confusion Matrix:
[[380   6   0]
 [  2  99   0]
 [  0   0  16]]


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

### Analisis Hasil Evaluasi Model Klasifikasi

**1. Perbandingan Hasil Evaluasi Sebelum dan Setelah Tuning:**

**Sebelum Tuning:**
- **Model Random Forest:**
  - **Accuracy:** 98.41%
  - **Precision untuk Cluster 0:** 0.99
  - **Precision untuk Cluster 1:** 0.94
  - **Precision untuk Cluster 2:** 1.00
  - **Recall untuk Cluster 0:** 0.98
  - **Recall untuk Cluster 1:** 0.98
  - **Recall untuk Cluster 2:** 1.00

  Model ini sudah menunjukkan performa yang sangat baik. Precision, recall, dan F1-score sangat seimbang untuk semua kelas, dengan model dapat mengenali Cluster 0 dan Cluster 1 dengan sangat baik, serta Cluster 2 dengan sempurna.

**Setelah Tuning:**
- **Model Random Forest (Tuning menggunakan GridSearchCV):**
  - **Accuracy:** 98.41%
  - **Precision untuk Cluster 0:** 0.99
  - **Precision untuk Cluster 1:** 0.94
  - **Precision untuk Cluster 2:** 1.00
  - **Recall untuk Cluster 0:** 0.98
  - **Recall untuk Cluster 1:** 0.98
  - **Recall untuk Cluster 2:** 1.00

  **Model Random Forest (Tuning menggunakan RandomizedSearchCV):**
  - **Accuracy:** 98.21%
  - **Precision untuk Cluster 0:** 0.99
  - **Precision untuk Cluster 1:** 0.94
  - **Precision untuk Cluster 2:** 1.00
  - **Recall untuk Cluster 0:** 0.98
  - **Recall untuk Cluster 1:** 0.97
  - **Recall untuk Cluster 2:** 1.00

  **Hasil:**
  - Tuning dengan **GridSearchCV** dan **RandomizedSearchCV** tidak memberikan peningkatan yang signifikan pada akurasi (98.41% menjadi 98.21%).
  - Namun, hasil tuning menunjukkan performa yang tetap sangat baik, dan model sudah hampir mencapai performa maksimal.
  - Kedua hasil tuning menunjukkan sedikit variasi pada nilai recall untuk Cluster 1, yang sedikit menurun dari 0.98 menjadi 0.97 di RandomizedSearchCV.

**2. Identifikasi Kelemahan Model:**

- **Precision dan Recall untuk Cluster 1:**
  - Pada **Logistic Regression**, terdapat masalah besar dengan Cluster 2, di mana tidak ada prediksi yang benar untuk kelas ini, menghasilkan nilai precision, recall, dan F1-score yang semuanya adalah 0.00. Ini mengindikasikan bahwa model tidak mampu menangani kelas minoritas (Cluster 2).
  - Pada **Random Forest**, meskipun akurasi dan metrik lainnya sangat tinggi, model sedikit kesulitan dalam memprediksi kelas 0 dan kelas 1, meskipun kesalahannya relatif kecil (6 untuk Cluster 0, 2 untuk Cluster 1).

- **Overfitting atau Underfitting:**
  - Model **Random Forest** tidak mengalami overfitting, karena akurasi pada data pelatihan sangat mirip dengan akurasi pada data uji (98.41% untuk training, 98.41% untuk testing).
  - Tidak ada indikasi **underfitting** pada Random Forest, karena model menunjukkan performa yang sangat baik pada semua metrik.
  - **Logistic Regression** menunjukkan kemungkinan **underfitting** karena akurasi dan kemampuan model sangat terbatas, terutama dalam mengklasifikasikan kelas minoritas (Cluster 2).

**3. Rekomendasi Tindakan Lanjutan:**

- **Jika Model Belum Memuaskan:**
  - **Kumpulkan Data Tambahan:** Jika masalah utama adalah kesulitan dalam mengklasifikasikan kelas minoritas (seperti Cluster 2 dalam Logistic Regression), menambah jumlah data untuk kelas minoritas dapat membantu model mempelajari pola lebih baik.
  - **Cobalah Algoritma Lain:** Jika Random Forest sudah memberikan hasil yang sangat baik tetapi masih ada kekurangan pada detail prediksi kelas minoritas, cobalah algoritma lain seperti **XGBoost** atau **LightGBM**, yang dapat memberikan hasil lebih baik dalam masalah klasifikasi yang tidak seimbang.
  - **Sampling Teknik untuk Kelas Minoritas:** Anda bisa mencoba **SMOTE (Synthetic Minority Over-sampling Technique)** untuk menangani ketidakseimbangan kelas.
  - **Ensemble Methods:** Kombinasi model **ensemble** yang berbeda dapat membantu meningkatkan prediksi kelas minoritas dengan cara yang lebih robust.

**Kesimpulan:**  
Model **Random Forest** menunjukkan performa yang sangat baik dengan akurasi yang hampir sempurna dan distribusi yang seimbang antara precision dan recall untuk semua kelas. Meskipun tuning dengan GridSearchCV dan RandomizedSearchCV tidak memberikan peningkatan signifikan, model sudah sangat baik. Sementara itu, **Logistic Regression** memiliki kekurangan besar dalam mengklasifikasikan kelas minoritas dan menghasilkan hasil yang jauh lebih rendah dibandingkan dengan Random Forest.